In [17]:
import torch
import os
import openai

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM

In [3]:
def get_encode_length(tokenizer, sentence) :
    encoded = tokenizer(sentence, padding=True, truncation=False)

    return len(encoded.input_ids)

In [4]:
def get_encode_data(tokenizer, sentence):
    encoded_inputs = tokenizer(sentence, padding=True, truncation=False)
    input_ids = torch.tensor(encoded_inputs['input_ids'])
    attention_masks = torch.tensor(encoded_inputs['attention_mask'])

    return input_ids, attention_masks

In [5]:
def summarize(input_text, tokenizer, model, device, max_tokens=200, min_tokens=100, num_beams=2) :
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Generate Summary Text Ids
    summary_text_ids = model.generate(
        input_ids=input_ids,
        bos_token_id=model.config.bos_token_id,
        eos_token_id=model.config.eos_token_id,
        length_penalty=2.0,
        max_new_tokens=max_tokens,
        min_new_tokens=min_tokens,
        no_repeat_ngram_size=3,
        num_beams=num_beams,
    )

    response = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)

    #print(len(input_text), len(response))

    return response

In [6]:
def get_model(model_name, device) :
    
    print(model_name)
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    

    # parallelization
    if torch.cuda.device_count() > 1:
        print(f'Using {torch.cuda.device_count()} GPUs.')

        model = torch.nn.DataParallel(model)

    return model.to(device), tokenizer

In [7]:
def run_summarize(sentence, models, max_tokens=200, min_tokens=100, num_beams=2) :
    for model in models :
        print('==== ', model[2], ' ===')
        response = summarize(sentence, model[1], model[0].module, device, max_tokens, min_tokens, num_beams)
        print(response, '\n')

In [8]:
os.environ['CURL_CA_BUNDLE'] = '/home/osung/Downloads/kisti_cert.crt'

In [8]:
sentence = "동진섬유는 글로벌 스포츠 브랜드 기업에 신발 원단을 공급하는 강소기업으로, 나이키, 아디다스의 세계 3대 핵심 협력업체 중 한 곳이다.\
 MBK파트너스는 연평균 10%에 가까운 전 세계 운동화 시장의 견조한 성장세는 물론, 전 세계 운동화 시장 점유율 45%를 차지하고 점유율을 확대해가고 \
있는 나이키 및 아디다스와 동진섬유와의 30년 이상 된 굳건한 협력 관계, 합성가죽보다는 섬유가 다양한 종류의 운동화에 지배적으로 사용되고 있는 \
추세 등을 긍정적으로 평가해 인수한 것으로 알려졌다."

In [9]:
#model_name = 'chihoonlee10/T3Q-ko-solar-dpo-v6.0'
#model_name = 'krevas/SOLAR-10.7B'
model_name = 'psyche/KoT5-summarization'
#model_name = 'ainize/kobart-news'
#model_name = 'noahkim/KoT5_news_summarization'
#model_name = 'KETI-AIR-Downstream/long-ke-t5-base-summarization'

In [10]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('available device: ', device)
else:
    device = torch.device("cpu")
    print('available device: ', device)

available device:  cuda


In [11]:
model_names = ['ainize/kobart-news', 'noahkim/KoT5_news_summarization', 'psyche/KoT5-summarization', 'KETI-AIR-Downstream/long-ke-t5-base-summarization']

In [12]:
models = []

In [13]:
for model_name in model_names :
    model, tokenizer = get_model(model_name, device)
    models.append((model, tokenizer, model_name))

ainize/kobart-news


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Using 2 GPUs.
noahkim/KoT5_news_summarization
Using 2 GPUs.
psyche/KoT5-summarization
Using 2 GPUs.
KETI-AIR-Downstream/long-ke-t5-base-summarization
Using 2 GPUs.


In [14]:
len(models)

4

In [25]:
summarize(sentence, models[0][1], models[0][0].module, device)

'MBK파트너스는 글로벌 스포츠 브랜드 기업에 신발 원단을 공급하는 강소기업인 동진섬유의 성장세와 확고한 협력 관계, 섬유가 다양한 종류의 운동화에 지배적으로 사용되고 있는 추세 등을 긍정적으로 평가해 인수한 것으로 알려졌다. MBK파트너스 관계자는 전 세계 운동화 시장의 견조한 성장세는 물론, 나이키 및 아디다스와 30년 이상 된 굳건한 협력관계, 합성가죽보다는 섬유의 다양한 종류에 지배적인 사용 추세 등 긍정적인 평가로 인수를 결정했다고 밝혔다.'

In [15]:
for model in models :
    print('==== ', model[2], ' ===')
    response = summarize(sentence, model[1], model[0].module, device)
    print(response, '\n')

====  ainize/kobart-news  ===
동진섬유는 글로벌 스포츠 브랜드 기업에 신발 원단을 공급하는 강소기업으로, 나이키, 아디다스의 세계 3대 핵심 협력업체 중 한 곳인데,  MBK파트너스는 연평균 10%에 가까운 전 세계 운동화 시장의 견조한 성장세는 물론,  30년 이상 된 굳건한 협력 관계, 합성가죽보다는 섬유가 다양한 종류의 운동화에 지배적으로 사용되고 있는 추세 등을 긍정적으로 평가해 동진 섬유를 인수한 것으로 알려졌다.   
 , . 꽁트 

====  noahkim/KoT5_news_summarization  ===
MBK파트너스는 연평균 10%에 가까운 전 세계 운동화 시장의 견조한 성장세는 물론 나이키, 아디다스와 동진섬유와의 30년 이상 된 굳건한 협력 관계 등을 긍정적으로 평가해 인수했으며 합성가죽보다는 섬유가 다양한.. 종류의.신발화에.지배적.으로.사용되고 있는 추세 등이 장점인 MCK 파트너스를 2020년까지 총 4개 계열사로 확대할 계획이라고 밝혔다. 

====  psyche/KoT5-summarization  ===
MBK파트너스는 글로벌 스포츠 브랜드 기업에 신발 원단을 공급하는 강소기업인 동진섬유의 성장세와 확고한 협력 관계, 섬유가 다양한 종류의 운동화에 지배적으로 사용되고 있는 추세 등을 긍정적으로 평가해 인수한 것으로 알려졌다. MBK파트너스 관계자는 전 세계 운동화 시장의 견조한 성장세는 물론, 나이키 및 아디다스와 30년 이상 된 굳건한 협력관계, 합성가죽보다는 섬유의 다양한 종류에 지배적인 사용 추세 등 긍정적인 평가로 인수를 결정했다고 밝혔다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===


/home/osung/anaconda3/lib/python3.9/site-packages/transformers/modeling_utils.py:838: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


동진섬유는 글로벌 스포츠 브랜드 기업에 신발 원단을 공급하는 강소기업으로, 나이키, 아디다스의 세계 3대 핵심 협력업체 중 한 곳이며, 30년 이상 된 굳건한 협력 관계, 섬유가 다양한 종류의 운동화에 지배적으로 사용되고 있는 추세 등을 긍정적 평가해 인수한 것으로 알려졌다.. 이들의 인수는 전 세계 운동화 시장의 성장세는 물론, 점유율을 확대해가는 중이다 



In [17]:
sentence = '정부가 19일 의과대학 정원 증원 규모를 조정할 수 있도록 해달라는 국립대 총장들의 건의를 적극 수용, 그간 고수했던 2000명 정원 축소에 나선다.\
이날 오후 정부서울청사에서 한덕수 국무총리 주재로 열리는 중앙재난안전대책본부(중대본) 회의에서 이같은 결정을 논의·발표할 예정이다. \
대통령실 고위 관계자는 이날 아시아경제에 "의대 증원 2000명에 대해서는 숫자에 매몰되지 않고 탄력적으로 생각을 할 수 있다"면서 \
"내년 입학전형 일정이 얼마 남지 않았기 때문에 빨리 결정을 해야 한다"고 밝혔다. \
앞서 강원대·경북대·경상국립대·충남대·충북대·제주대 등 6개 거점 국립대 총장은 전날 대학별로 의대 증원분의 50~100% 범위에서 자율적으로 \
2025학년도 신입생을 모집할 수 있게 해달라는 건의문을 교육부에 보냈다. 자율성을 보장해달라는 의견으로 사실상 축소 가능성을 열어달라는 의견인 \
것으로 풀이된다. 이에 대해 대통령실 관계자는 "의대가 있는 국립대 총장들이 합리적으로 생각해볼 수 있는 안을 가져온 것이니 긍정적으로 검토하고 \
있다"면서 "자율성을 드리는 형태로 의대증원의 취지는 살릴 수 있는 안"이라고 말했다. 특히 이 관계자는 이달 말까지 대학이 내년 대입전형 시행계획을 \
한국대학교육협의회에 제출해야 하는 상황으로 시일이 촉박한 만큼 빠른 결정이 필요하다는 입장을 표명했다. 의정 갈등이 해법을 모색하지 못하는 상황이 \
지속될 경우 수험생 혼란만 커질 수 있기 때문이다. 대통령실 관계자는 "(대입전형 시행계획) 마지노선이 상당히 가까워지고 있고 지금이 \
(의대증원 규모를) 무리 없이 조정할 수 있는 시기이기 때문에 (더 늦기 전에) 빨리 해드리긴 해야한다"면서 "중대본에서 결정할 것으로 보인다"고 \
설명했다. 이어 "의료계에 단일안을 계속 요청했지만 0명 혹은 원점 재논의를 요청하는 상황이라 이를 수용할 수는 없다"면서 "이런 시점에서 국립대 \
총장들이 합리적인 안을 가져온 것이므로 긍정적으로 생각한다"고 덧붙였다.'

In [18]:
run_summarize(sentence, models)

====  ainize/kobart-news  ===
정부가과대학 정원 증원 규모를 조정할 수 있도록 해달라는 국립대 총장들의 건의를 적극 수용, 그간 고수했던 2000명 정원 축소에 나선 대통령실 고위 관계자는 이날 아시아경제에 탄력적으로 생각을 할 수 있다며  "내년 입학전형 일정이 얼마 남지 않았기 때문에 빨리 결정을 해야 한다"고 밝혔고 특히 이 관계자는 이달 말까지 대학이 내년 대입전형 시행계획을 한국대학교육협의회에 제출해야 하는 상황으로 시일이 촉박한 만큼 빠른 결정이 필요하다는 입장을 표명 표명했통신다. 

====  noahkim/KoT5_news_summarization  ===
대통령실 고위 관계자는 19일 의과대학 정원 증원 규모를 조정할 수 있도록 해달라는 국립대 총장들의 요청을 적극 수용, 그간 고수했던 2000명 인원 축소에 나서며 이날 오후 정부서울청사에서 한덕수 국무총리 주재로 열리는 중앙재난안전대책본부(중대본) 회의에서이같은 결정을논의·발표하고 내년 입학전형 일정이 얼마 남지 않은 만큼 빨리 결단을해야 한다고 밝혔다. 

====  psyche/KoT5-summarization  ===
정부가 19일 의과대학 정원 증원 규모를 조정할 수 있도록 해달라는 국립대 총장들의 건의를 적극 수용, 그간 고수했던 2000명 정원 축소에 나서며 이날 오후 정부서울청사에서 한덕수 국무총리 주재로 열리는 중앙재난안전대책본부(중대본) 회의에서 이같은 결정을 논의·발표할 예정이다. 대통령실 고위 관계자는 이날 아시아경제에 "의대 증원에 대해서는 숫자에 매몰되지 않고 탄력적으로 생각을 할 수 있다"면서 "내년 입학전형 일정이 얼마 남지 않았기 때문에 빨리 결정을 해야 한다"고 밝혔다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
정부가 19일 의과대학 정원 증원 규모를 조정할 수 있도록 해달라는 국립대 총장들의 건의를 적극 수용, 그간 고수했던 2000명 정원 축소에 나서며, 대통령실

In [15]:
sentence = " 고금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. \
특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 \
지수 하락 폭을 키우고 있다. 코스피지수는 이날 오전 11시 4분 기준 2566.77이다. 전날보다 67.93포인트(2.58%) 하락했다. \
지난 1월 이후 가장 낮은 수준이다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. \
개인만 9123억원 순매수 중이다. 유가증권시장 대장주인 삼성전자와 SK하이닉스가 각각 3%, 5% 이상 약세를 보이고 있다. \
업종 중에서도 반도체와 반도체장비의 하락률이 3.8%로 컸다. TSMC가 전날 올해 1분기 실적 발표 후 컨퍼런스콜에서 올해 메모리 반도체를 제외한 \
전체 반도체 시장 성장률 전망치를 기존 ‘10% 이상’에서 10%로 낮춘 영향으로 보인다. 김석환 미래에셋증권 연구원은 “TSMC가 반도체 업황을 하향 \
조정한 여파로 밤사이 미국 증시에서 반도체 관련 종목군의 약세가 뚜렷했다”며 “한국 증시에서도 관련 종목군의 주가 약세가 지수 하락을 주도하고 있다” \
고 했다. 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다. 전날 상승분을 모두 반납했다. 코스닥시장에서도 외국인과 기관이 \
3185억원, 330억원을 순매도하고 있다. 개인만 3577억원 매수 우위를 보이고 있으나 지수를 방어하기엔 벅찬 상황이다. 에코프로비엠을 비롯한 \
코스닥시장 시가총액 상위 종목 주식 대부분이 전날보다 낮은 가격에 거래 중이다. 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 \
구두개입으로 주춤했던 환율도 하루 만에 반등했다. 서울 외환시장에서 미국 달러 대비 원화(원·달러) 환율은 이날 오전 1390원선을 다시 넘어섰다. \
중동 정세 불안으로 국제 유가도 뛰었다. 이스라엘이 계획 중인 가자지구 남단 라파 지상전을 두고 미국 백악관은 민간인 피해 가능성을 여전히 \
우려하지만, 팔레스타인 무장 정파 하마스를 격퇴한다는 공동의 목표에 합의했다고 발표했다. 이어 이스라엘이 이란 본토를 미사일로 공격했다는 보도도 \
나왔다. 미국 서부텍사스원유(WTI) 5월물과 브렌트유 6월물 모두 현재 3%대 오름세를 보이고 있다." 

In [16]:
run_summarize(sentence, models)

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 이에 따라 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 코스피·코스닥지수가 약세를 보이고 있으며, 특히 고금리·고환율·저유가 등 이른바 3고 우려가 다시 불거지면서 19일장중 코스피와 코스닥지수는 약세 폭을 키우고 있다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸고 유가증권시장 대장주인 삼성전자와SK하이닉스가 각각 3%, 5% 이상 약세를 나타내고 있다. 코스피지수는 이날 오전 11시 4분 기준 2566.77이다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===


/home/osung/anaconda3/lib/python3.9/site-packages/transformers/modeling_utils.py:838: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있으며, 특히 코스피지수는 이날 오전 11시 4분 기준 2566.77로 전날보다 67.93포인트(2.57%) 하락한 것으로 나타났다.. 김석환 미래에셋증권 연구원은 “TSMC의 반도체 업황 하향 조정 여파로 밤사이 미국 증시에서 반도체 관련 종목군의 약세가 뚜렷했다”며 “한국 증시에서도 관련 종목 군의 주가 약세가 지수 하락을 주도하고 있다” 고 했다. 



In [22]:
run_summarize(sentence, models)

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다.   
중동 정세 불안으로 국제 유가도 뛰었다.. . 이스라엘이 계획 중인 가자지구 남단 라파 지상전을 두고 미국 백악관은 민간인 피해 가능성을 여전히 우려하지만, 팔레스타인 무장 정파 하마스를 격퇴한다는 공동의 목표에 합의했다고 발표했다. 
.이루워즈(왼쪽(왼쪽 전력 전력에 베트남이 이란 본토를 미사일로 공격했다는 보도도 나왔다. 

이 방송뉴스를 삭제하라고 주장했다.(왼쪽목소리란에 게재했다고 보도했다.

. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 이른바 3고 우려로 인해 코스피지수는 전날보다 67.93포인트(2.58%) 하락하며 약세를 보이고 있으며 이는 지난 1월 이후 가장 낮은 수준으로 유가증권시장에서 기관과 외국인이 각각 7712억원 ‘팔자'에 나서며 지수 상승을 주도하고 있는 가운데 코스닥시장 시가총액 상위 종목 주식 대부분은 이날 낮 12시 4분 기준 2566.76이다. 이에 대해 김석환 미래에셋증권 연구원은 미국 달러 대비 원화(원·달러)의 환율은,전날 오전 1390원선이 다시 넘어섰다고 밝혔다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있

In [24]:
run_summarize(sentence, models)

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 이에 따라 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 이른바 3고 우려로 인해 코스피·코스닥지수는 약세를 보이고 있으며, 유가증권시장에서 기관과 외국인이 각각 6712억원 ‘팔자'에 나서며 지수 하락 폭을 키우고 있고 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18으로 이날 오전 11시 4분 기준 2566.77이다. 이에 대해 김석환 미래에셋증권 연구원은 글로벌 금융위기 이후 가장 낮은 수준의 증시 변동성이 지속되고 있다고 분석했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸고, 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다. 중동 정세 불안으로 국제 유가도 뛰었다. 이스라엘이 계획 중인 가자지구 남단 라파 지상전을 두고 미국 백악관은 민간인 피해 가능성을 여전히 우려하지만, 팔레스타인 무장 정파 하마스를 격퇴한다는 공동의 목표에 합의했다고 발표했다. 

====  KETI-AIR-Downstre

In [28]:
run_summarize(sentence, models) # num_beams=2

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다.   
중동 정세 불안으로 국제 유가도 뛰었다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 이른바 3고(高) 우려로 인해 코스피·코스닥지수가, 전날보다 67.93포인트 하락하며 약세를 보이고 있다. 이에 대해 김석환 미래에셋증권 연구원은 “TCM이 올해 1분기 실적 발표 후 컨퍼런스콜에서 올 메모리 및 낸드플래시 시장 전체 경기 둔화 가능성을 반영해 글로벌 증시 변동성이 확대될 것으로 예상된다 고 말했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 코스피지수는 이날 오전 11시 4분 기준 2566.77이며 전날보다 67.93포인트(2.58%) 하락했다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등의 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다.. 김석환 미래에셋증

In [32]:
run_summarize(sentence, models, num_beams=4)

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다.   
중동 정세 불안으로 국제 유가도 뛰었다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 이른바 3고 우려로 인해 코스피지수는 전날보다 67.93포인트(2.58%) 하락하며 약세를 보이고 있으며 이는 지난 1월 이후 가장 낮은 수준이다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸고, 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다. 중동 정세 불안으로 국제 유가도 뛰었다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스의 주가가 꺾이면서 코스피지수가 전날보다 67.93포인트 하락하며 약세를 보이고 있는 가운데, 김석환 미래에셋증권 연구원은 TSCM가 하향 조정한 여파로 밤사이 미국 증시에서 관련 종목군의 약세가 뚜렷했다며 한국 증시에서도 관련종목의 주가의

In [33]:
run_summarize(sentence, models, num_beams=10)

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 이에 따라 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추는 등 이른바 3고(高) 우려로 인해 코스피지수는 전날보다 67.93포인트 하락하여 지난 1월 이후 가장 낮은 수준을 보이고 있으며 유가증권시장 대장주인 삼성전자와 SK하이닉스의 주가 약세도 지수를 방어하기엔 벅찬 상황이다. 이에 김석환 미래에셋증권 연구원은 “글로벌 경기 둔화 가능성이 커지고 있는 가운데 미국 달러 대비 원화 환율은 이날 오전 1390원선에서 다시 반등했다"고 말했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸고, 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다. 중동 정세 불안으로 국제 유가도 뛰었다. 이스라엘이 계획 중인 가자지구 남단 라파 지상전을 두고 미국 백악관은 민간인 피해 가능성을 여전히 우려하지만, 팔레스타인 무장 정파 하마스를 격퇴한다는 공동의 목표에 합의했다고 발표했다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summari

In [46]:
def summarize2(input_text, tokenizer, model, device, max_tokens=200, min_tokens=100, num_beams=2) :
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Generate Summary Text Ids
    summary_text_ids = model.generate(
        input_ids=input_ids,
        bos_token_id=model.config.bos_token_id,
        eos_token_id=model.config.eos_token_id,
        length_penalty=2.0,
        max_new_tokens=max_tokens,
        min_new_tokens=min_tokens,
        #no_repeat_ngram_size=3,
        num_beams=num_beams,
        num_return_sequences=2
    )

    response = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)

    #print(len(input_text), len(response))

    return response

In [47]:
summarize2(sentence, models[0][1], models[0][0].module, device, num_beams=8)

'파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고 업종 중에서도 반도체와 반도체장비의 하락률이 3.8%로 컸다.'

In [51]:
run_summarize(sentence, models, num_beams=10)

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고 업종 중에서도 반도체와 반도체장비의 하락률이 3.8%로 컸다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 코스피·코스닥지수가 약세를 보이고 있으며 특히 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있다. 

====  psyche/KoT5-summarization  ===
고금리·고환율·고유가 등 이른바 3고() 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있으며 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 김석환 미래에셋증권 연구원은 TSCM가 반도체 업황을 하향 조정한 여파로 밤사이 미국 증시에서 반도체 관련 종목군의 약세가 뚜렷했다며 한국 증시에서도 관련 종목군의 주가 약세가 지수 하락을 주도하고 있다고 했다. 김석환 미래에셋증권 연구원은 TSCM가 반도체 업황을 하향 조정한 여파로 밤사이

In [65]:
run_summarize(sentence, models, max_tokens=300, min_tokens=200, num_beams=2)

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다.   
중동 정세 불안으로 국제 유가도 뛰었다..  .달러달러(WTI) 5월물과 브렌트유(WTC) 5월물 모두 모두 현재 3%대 오름세를 보이고 있다...했습니다.했습니다.(왼쪽(왼쪽(왼쪽) 팔레스타인 무장 정파 하마스를 격퇴한다는 공동의 목표에 합의했다고 발표했다....달러 환율은 이날 오전 1390원선을 다시 넘어서서 오전 13 90원선을 회복하였습니다...처분이름.이스라엘이 계획 중인 가자지구 남단 라파 지상전을 두고 미국 백악관은 민간인 피해 가능성을 여전히 우려하지만, 팔레스타단 무장 정지하지하철도를 격퇴한다고 공동의목표에 합의했다라고 발표했다고 발표하였다.. 

====  noahkim/KoT5_news_summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있으며, 특히 세계 최고 파운드리아 회사인 대만의 반도체·반도체장비 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있다. 김석환 미래에셋증권 연구원은 “TSCM가 올해 1분기 실적 발표 후 컨퍼런스콜에서 올해는 메모리 반도체를 제외한 전체 반도체시장 성장률 전망치가 기존 '10% 이상’에서 10%를 낮춘 영향으로 보인다”고 말했다. 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다. 서울 외환시장에서 미국 달러 대비 원화(원·달러) 환율은 이날 오전 1390원선을 다시 넘어섰다. 뉴욕 

In [77]:
def summarize(input_text, tokenizer, model, device, max_tokens=200, min_tokens=100, num_beams=2) :
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Generate Summary Text Ids
    summary_text_ids = model.generate(
        input_ids=input_ids,
        bos_token_id=model.config.bos_token_id,
        eos_token_id=model.config.eos_token_id,
        length_penalty=2.0,
        max_new_tokens=max_tokens,
        min_new_tokens=min_tokens,
        no_repeat_ngram_size=3,
        num_beams=num_beams,
        do_sample=False #True
    )

    response = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)

    #print(len(input_text), len(response))

    return response

In [68]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=4)

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다.   
중동 정세 불안으로 국제 유가도 뛰었다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 코스피·코스닥지수가 약세를 보이고 있는데 특히 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 김석환 미래에셋증권 연구원은 TSCM가 반도체 업황을 하향 조정한 여파로 밤사이 미국 증시에서 반도체 관련 종목군의 약세가 뚜렷했다고 했다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
19일 장 중 코스피·코스닥지수가 약세를 보이고 있는데, 특히 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며, 코스닥시장에서도 외국인과 기관이 3185억원, 330억원을 

In [69]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=8) # do_sample = True

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다.  
이날 오전에 미국 서부텍사스원유(WTI) 5월물과 브렌트유 6월물 모두 모두 현재 3%대 오름세를 보이고 있다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 올해 메모리 반도체를 제외한 전체 반도체 시장 성장률 전망치를 기존 ‘10% 이상’에서 10%로 낮춘 영향으로 반도체와 반도체장비의 하락률이 3.8%로, 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등하는 등 이른바 3고 우려가 다시 불거지면서 코스피·코스닥지수가 약세를 보이고 있다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 김석환 미래에셋증권 연구원은 TSCM가 반도체 업황을 하향 조정한 여파로 밤사이 미국 증시에서 반도체 관련 종목군의 약세가 뚜렷했다며 한국 증시에서도 관련종목군의 주가 약세가 지수 하락을 주도하고 있다고 했다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
19일 장 중 코스피·코스닥지수가 약세를 보이고 있는

In [72]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=8) # do_sample = False

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록하며 전날 상승분을 모두 반납했다. 

====  noahkim/KoT5_news_summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 올해 메모리 반도체를 제외한 전체 반도체 시장 성장률 전망치를 기존 ‘10% 이상’에서 10%로 낮춘 영향으로 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데, 김석환 미래에셋증권 연구원은 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 김석환 미래에셋증권 연구원은 TSCM가 반도체 업황을 하향 조정한 여파로 밤사이 미국 증시에서 반도체 관련 종목군의 약세가 뚜렷했다며 한국 증시에서도 관련종목군의 주가 약세가 지수 하락을 주도하고 있다고 했다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있

In [74]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=8) # do_sample = False

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고 업종 중에서도 반도체와 반도체장비의 하락률이 3.8%로 컸다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 코스피·코스닥지수가 약세를 보이고 있으며 특히 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 김석환 미래에셋증권 연구원은 TSCM가 반도체 업황을 하향 조정한 여파로 밤사이 미국 증시에서 반도체 관련 종목군의 약세가 뚜렷했다며 한국 증시에서도 관련 종목군의 주가 약세가 지수 하락을 주도하고 있다고 했다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스

In [76]:
run_summarize(sentence, models, max_tokens=300, min_tokens=200, num_beams=3) # do_sample = False, no_repeat = None

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다.   
중동 정세 불안으로 국제 유가도 뛰었다. 

중장 중 이스라엘이 계획 중인 가자지구 남단 라파 지상전을 두고 미국 백악관은 민간인 피해 가능성을 여전히 우려하지만, 팔레스타인 무장 정파 하마스를 격퇴한다는 공동의 목표에 합의했다고 발표했다. 
이스라엘이 이란 본토를 미사일로 공격했다는 보도와 이스라엘이 이란 본토를 미사일로 공격했다는 보도와 이스라엘이 이란 본토를 미사일로 공격했다는 보도와  이스라엘이 이란 본토를 미사일로 공격했다는 보도와  이스라엘이 이란 본토를 미사일로 공격했다는일(현지일(현지 토지 정프 하마스를 격퇴한다는 공동의 목표에 합의했다고 발표했다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체 업종 주가가 꺾이면서 코스피·코스닥지수가 약세를 보이고 있으며 특히 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했으며 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다. 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다. 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만

In [78]:
run_summarize(sentence, models, max_tokens=300, min_tokens=200, num_beams=3) # do_sample = False, no_repeat = 3

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다.   
중동 정세 불안으로 국제 유가도 뛰었다. 

중장 중 이스라엘이 계획 중인 가자지구 남단 라파 지상전을 두고 미국 백악관은 민간인 피해 가능성을 여전히 우려하지만, 팔레스타인 무장 정파 하마스를 격퇴한다는 공동의 목표에 합의했다고 발표했다. 
. 이어 이스라엘이 이란 본토를 미사일로 공격했다는 보도다. 

이 이스라엘이 할수 있다는 보도에 삽청정도가 예상이 현실화된  내용의   이스라엘이 중동 정세 불안이 발생했다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 코스피·코스닥지수가 약세를 보이고 있으며, 특히 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등한 것으로 나타났다. 이에 따라 코스피지수는 이날 오전 11시 4분 기준 2566.77을 기록했다. 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 나타냈다. 반면 코스닥시장 시가총액 상위 종목 주식 대부분이 전날보다 낮은 가격에 거래 중이다. 한국 증시에서도 관련 종목군의 주가 약세가 지수 하락을 주도하고 있다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이

In [79]:
run_summarize(sentence, models, max_tokens=200, min_tokens=100, num_beams=3) # do_sample = False, no_repeat = 3

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다.   
중동 정세 불안으로 국제 유가도 뛰었다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 코스피·코스닥지수가 약세를 보이고 있으며, 특히 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데, 김석환 미래에셋증권 연구원은 TSCM가 반도체 업황을 하향 조정한 여파로 밤사이 미국 증시에서 반도체 관련 종목군의 약세가 뚜렷했다며 한국 증시에서도 관련종목군의 주가 약세가 지수 하락을 주도하고 있다고 했다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있으며, 특히 코스피지수는 이날 오전 11시 4분 기준 2566.77로 전날보다 67.93포인트(2.57%) 하락한 것으로 나타나 지난 1월 이

In [80]:
run_summarize(sentence, models, max_tokens=200, min_tokens=100, num_beams=4) # do_sample = False, no_repeat = 3

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다.   
중동 정세 불안으로 국제 유가도 뛰었다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체업종 주가가 꺾이면서 코스피·코스닥지수가 약세를 보이고 있으며 특히 고금리·고환율,고유가 등의 이른바 3고 우려가 다시 불거지면서 18일 이후 가장 낮은 수준으로 하락 폭을 키우고 있는 가운데 코스피지수는 이날 오전 11시 4분 기준 2566.77로 전날보다 67.93포인트(2.58%) 하락했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데, 김석환 미래에셋증권 연구원은 TSCM가 반도체 업황을 하향 조정한 여파로 밤사이 미국 증시에서 반도체 관련 종목군의 약세가 뚜렷했다며 한국 증시에서도 관련종목군의 주가 약세가 지수 하락을 주도하고 있다고 했다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있으며, 특히 코스피

In [82]:
run_summarize(sentence, models, max_tokens=300, min_tokens=200, num_beams=4) # do_sample = False, no_repeat = 3

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다.   
이날 오전 1390원선을 다시 넘어섰다고 밝혔다. 
이들은 중동 정세 불안으로 국제 유가도 뛰었다. 

또 이스라엘이 계획 중인 가자지구 남단 라파 지상전을 두고 미국 백악관은 민간인 피해 가능성을 여전히 우려하지만, 팔레스타인 무장 정파 하마스를 격퇴한다는 공동의 목표에 합의했다고 발표했다. 
이번에는 이스라엘이 이란 본토를 미사일로 공격했다는 보도도 나왔다. 
(왼쪽(왼쪽(왼쪽 본드 유전자는 방송뉴스에 게재했다고 한다) 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체업종 주가가 꺾이면서 코스피·코스닥지수가 약세를 보이고 있으며 특히 고금리·고환율,고유가 등의 이른바 3고 우려가 다시 불거지면서 18일 이후 가장 낮은 수준으로 하락 폭을 키우고 있는 가운데 코스피지수는 이날 오전 11시 4분 기준 2566.77로 전날보다 67.93포인트(2.58%) 하락한 것으로 집계됐다. 김석환 미래에셋증권 연구원은 “TSCM가 올해 1분기 실적 발표 후 컨퍼런스콜에서 올해는 메모리 반도체를 제외한 전체 반도체시장 성장률 전망치 기존 '10% 이상’에서 10%로 낮춘 영향으로 보인다”고 말했다. 같은 시각 코스닥지수도 24.47포인트 내린 831.18을 기록했다. 서울 외환시장에서 미국 달러 대비 원화(원·달러)  

====  psyche/KoT5-summarization  ==

In [89]:
run_summarize(sentence, models, max_tokens=400, min_tokens=120, num_beams=4) # do_sample = False, no_repeat = 3

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다.   
이날 오전 1390원선을 다시 넘어섰다고 밝혔다. 
이들은 중동 정세 불안으로 국제 유가도 뛰었다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체업종 주가가 꺾이면서 코스피·코스닥지수가 약세를 보이고 있으며 특히 고금리·고환율,고유가 등의 이른바 3고 우려가 다시 불거지면서 18일 이후 가장 낮은 수준으로 하락 폭을 키우고 있는 가운데 코스피지수는 이날 오전 11시 4분 기준 2566.77로 전날보다 67.93포인트(2.58%) 하락했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데, 김석환 미래에셋증권 연구원은 TSCM가 반도체 업황을 하향 조정한 여파로 밤사이 미국 증시에서 반도체 관련 종목군의 약세가 뚜렷했다며 한국 증시에서도 관련종목군의 주가 약세가 지수 하락을 주도하고 있다고 했다. 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 

In [99]:
def summarize(input_text, tokenizer, model, device, max_tokens=200, min_tokens=100, num_beams=2) :
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Generate Summary Text Ids
    summary_text_ids = model.generate(
        input_ids=input_ids,
        bos_token_id=model.config.bos_token_id,
        eos_token_id=model.config.eos_token_id,
        length_penalty=2.0,
        max_new_tokens=max_tokens,
        min_new_tokens=min_tokens,
        no_repeat_ngram_size=3,
        num_beams=num_beams,
        top_p=0.95,
        top_k=0,
        do_sample=False #True
    )

    response = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)

    #print(len(input_text), len(response))

    return response

In [95]:
run_summarize(sentence, models, max_tokens=400, min_tokens=200, num_beams=4) # do_sample = False, no_repeat = 3

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다.   
이날 오전 1390원선을 다시 넘어섰다고 밝혔다. 
이들은 중동 정세 불안으로 국제 유가도 뛰었다. 

또 이스라엘이 계획 중인 가자지구 남단 라파 지상전을 두고 미국 백악관은 민간인 피해 가능성을 여전히 우려하지만, 팔레스타인 무장 정파 하마스를 격퇴한다는 공동의 목표에 합의했다고 발표했다. 
이번에는 이스라엘이 이란 본토를 미사일로 공격했다는 보도도 나왔다. 
(왼쪽(왼쪽(왼쪽 본드 유전자는 방송뉴스에 게재했다고 한다) 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체업종 주가가 꺾이면서 코스피·코스닥지수가 약세를 보이고 있으며 특히 고금리·고환율,고유가 등의 이른바 3고 우려가 다시 불거지면서 18일 이후 가장 낮은 수준으로 하락 폭을 키우고 있는 가운데 코스피지수는 이날 오전 11시 4분 기준 2566.77로 전날보다 67.93포인트(2.58%) 하락한 것으로 집계됐다. 김석환 미래에셋증권 연구원은 “TSCM가 올해 1분기 실적 발표 후 컨퍼런스콜에서 올해는 메모리 반도체를 제외한 전체 반도체시장 성장률 전망치 기존 '10% 이상’에서 10%로 낮춘 영향으로 보인다”고 말했다. 같은 시각 코스닥지수도 24.47포인트 내린 831.18을 기록했다. 서울 외환시장에서 미국 달러 대비 원화(원·달러) 환율은 전날 오후 1390원선을 또 넘어섰다. 뉴욕 증시에서 외국인과

In [97]:
run_summarize(sentence, models, max_tokens=300, min_tokens=200, num_beams=4) # do_sample = False, no_repeat = 3

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다.   
이날 오전 1390원선을 다시 넘어섰다고 밝혔다. 
이들은 중동 정세 불안으로 국제 유가도 뛰었다. 

또 이스라엘이 계획 중인 가자지구 남단 라파 지상전을 두고 미국 백악관은 민간인 피해 가능성을 여전히 우려하지만, 팔레스타인 무장 정파 하마스를 격퇴한다는 공동의 목표에 합의했다고 발표했다. 
이번에는 이스라엘이 이란 본토를 미사일로 공격했다는 보도도 나왔다. 
(왼쪽(왼쪽(왼쪽 본드 유전자는 방송뉴스에 게재했다고 한다) 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체업종 주가가 꺾이면서 코스피·코스닥지수가 약세를 보이고 있으며 특히 고금리·고환율,고유가 등의 이른바 3고 우려가 다시 불거지면서 18일 이후 가장 낮은 수준으로 하락 폭을 키우고 있는 가운데 코스피지수는 이날 오전 11시 4분 기준 2566.77로 전날보다 67.93포인트(2.58%) 하락한 것으로 집계됐다. 김석환 미래에셋증권 연구원은 “TSCM가 올해 1분기 실적 발표 후 컨퍼런스콜에서 올해는 메모리 반도체를 제외한 전체 반도체시장 성장률 전망치 기존 '10% 이상’에서 10%로 낮춘 영향으로 보인다”고 말했다. 같은 시각 코스닥지수도 24.47포인트 내린 831.18을 기록했다. 서울 외환시장에서 미국 달러 대비 원화(원·달러)  

====  psyche/KoT5-summarization  ==

In [98]:
run_summarize(sentence, models, max_tokens=300, min_tokens=200, num_beams=20) # do_sample = False, no_repeat = 3

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있어 코스피지수는 이날 오전 11시 4분 기준 2566.77로 전날보다 67.93포인트(2.58%) 하락해 지난 1월 이후 가장 낮은 수준이라고 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서면서, 개인만 9123억원 순매도에 나서며 지수 하락 하락에 한 몫을 하고 있는 상황이라고 전했다. 

이번에는 김석환 미래에셋증권 연구원은 “TSCM가 반도체 업황을 하향 조정한 여파로 밤사이 미국 증시에서 반도체 관련 종목군의 약세가 뚜렷했다”며 “한국 증시에서도 관련 종목 군의 주가 약세가 지수 하락을 주도하고 있다” 고 말했다 

====  noahkim/KoT5_news_summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 올해 메모리 반도체를 제외한 전체 반도체 시장 성장률 전망치를 기존 ‘10% 이상’에서 10%로 낮춘 영향으로 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등하는 등 이른바 3고 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 이에 따라 코스피지수는 이날 오전 11시 4분 기준 2566.77로 전날보다 67.93포인트(2.58%) 하락했다. 이는 지난 1월 이후 가장 낮은 수준이다. 한국 증시에서도 관련 종목군의 주가 약세가 지수 하락을 주도하고 있다고 김 미래에셋증권 연구원은 말했다. 반면 코스닥시장에서도 외국인과 기관이 3185억원, 330억원을 순매도하고 있으며 개인만 3577억원 매수 우위를 보이며 지수를 방어하기엔 벅찬 

====  psyche

In [100]:
run_summarize(sentence, models, max_tokens=200, min_tokens=100, num_beams=20) # do_sample = False, no_repeat = 3

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 이에 따라 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 

====  noahkim/KoT5_news_summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 올해 메모리 반도체를 제외한 전체 반도체 시장 성장률 전망치를 기존 ‘10% 이상’에서 10%로 낮춘 영향으로 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등하는 등 이른바 3고 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 

====  psyche/KoT5-summarization  ===
고금리·고환율·고유가 등 이른바 3고() 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 김석환 미래에셋증권 연구원은 TSCM가 반도체 업황을 하향 조정한 여파로 밤사이 미국 증시에서 반도체 관련 종목군의 약세가 뚜렷했다며 한국 증시에서도 관련종목군의 주가 약세가 지수 하락을 주도하고 있다고 했다. 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summariza

In [101]:
run_summarize(sentence, models, max_tokens=200, min_tokens=100, num_beams=10) # do_sample = False, no_repeat = 3

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록하며 전날 상승분을 모두 반납했다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 올해 메모리 반도체를 제외한 전체 반도체 시장 성장률 전망치를 기존 ‘10% 이상’에서 10%로 낮추는 등 이른바 3고 우려가 다시 불거지면서 코스피·코스닥지수가 약세를 보이고 있으며 유가증권시장에서도 외국인과 기관이 3185억원, 330억원을 순매도하고 있고 코스닥시장 시가총액 상위 종목 주식 대부분이 전날보다 낮은 가격에 거래 중이며 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다. 

====  psyche/KoT5-summarization  ===
고금리·고환율·고유가 등 이른바 3고() 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있으며 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 김석환 미래에셋증권 연구원은 TSCM가 반도체 업황을 하향 조정한 여파로 밤사이 미국 증시에서 반도체 관련 종목군의 약세가 뚜렷했다며 한국 증시에서도 관련종목군의 주가 약세가 지수 하락을 주도하고 있다고 했다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summ

In [102]:
run_summarize(sentence, models, max_tokens=200, min_tokens=100, num_beams=1) # do_sample = False, no_repeat = 3

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다.    
 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 코스피·코스닥지수가 고금리·고환율·저유가 등 이른바 3고 우려가 다시 불거지면서 약세를 보이고 있으며 특히 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등의 반도체업종 주가가 꺾이면서 지수 하락 폭을 키우고 있다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 김석환 미래에셋증권 연구원은 TSCM가 반도체 업황을 하향 조정한 여파로 밤사이 미국 증시에서 반도체 관련 종목군의 약세가 뚜렷했다며 한국 증시에서도 관련 종목군 주가 약세가 지수 하락을 주도하고 있다고 했다. 코스피지수는 이날 오전 11시 4분 기준 2566.77이다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
19일 장 중 코스피·코스닥지수가 약세를 보이고 있는데, 특히 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며, 김석환 미래에셋증권 연구원은 “TSCM가 반반도체 업황을 하향 조정한 여파로 밤사이 미국 증시에서 반도체 관련 종

In [107]:
run_summarize(sentence, models, max_tokens=300, min_tokens=200, num_beams=4) # do_sample = False, no_repeat = 3

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 이에 따라 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내린 것으로 나타났다.    
이날 오전 11시 4분 기준 2566.77 포인트로 전날보다 67.93포인트(2.58%) 하락했다. 
김석환 미래에셋증권 연구원은 “TSMC가반도체 업황을 하향 조정한 여파로 밤사이 미국 증시에서 반도체 관련 종목군의 약세가 뚜렷했다”며 “한국 증시에서도 관련 종목구의 주가 약세가 지수 하락을 주도하고 있다” 고 말했다.
이기는 하지만  “TSTS가 반도체 업언을 통해 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다.
(왼쪽(왼쪽(왼쪽) 환율은 이날 오전 1390원선을 다시 넘어섰다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체업종 주가가 꺾이면서 코스피·코스닥지수가 약세를 보이고 있으며 특히 고금리·고환율,고유가 등의 이른바 3고 우려가 다시 불거지면서 18일 이후 가장 낮은 수준으로 하락 폭을 키우고 있는 가운데 코스피지수는 이날 오전 11시 4분 기준 2566.77로 전날보다 67.93포인트(2.58%) 하락한 것으로 집계됐다. 김석환 미래에셋증권 연구원은 “반도체 업황을 하향 조정한 여파로 밤사이 미국 증시에서 반도체와 반도체장비의 하락률이 3.8%로 컸다”고 말했다. 같은 시각 코스닥지수도 24.47포인트 내린 831.18을 기록했다. 코스닥시장 시가총액 상위 

In [108]:
run_summarize(sentence, models, max_tokens=300, min_tokens=200, num_beams=1) # do_sample = False, no_repeat = 3

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 세계 최대파운드리 업체인 TSSMC의 반도체 성장률 전망치 하락으로 삼성전자와 SK텔레콤 등 반도체 관련 주가가 꺾이며 지수 하락을 키운 것으로 나타났다.    

﻿﻿
﻿손해율(高)에 대한  4%(高)에서  TSH가 반도체 업계를 하향 조정한 여파로 밤사이 미국 증시에서 반도체 관련 종목군의 약세가 뚜렷했는데, “한국 증시에서도 관련 종목군군의 주가 약세가 지수 하락 주를 주도하고 있다” 고 했다. 

김석환 미래에셋증권 연구원은 말했다. 


김약환 미래에셋증권은 말했다. 
자세한 목표에 맞춰 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 코스피·코스닥지수가 약세를 보이고 있으며 특히 세계적 반도체·전자 업체 TV 시장 점유율 1위인 삼성전자와 SK하이닉스 등 반도체업종 주가가 꺾이면서 지수 하락 폭을 키우고 있다. 김석환 미래에셋증권 연구원은 “TSmC가, 반도체가 업황을 하향 조정한 여파로 밤사이 미국 증시에서 반도체, 장비 등의 약세가 뚜렷했다”며 �‘한국 증시에서도 관련 종목군의 주가 약세 지수가,지수 상승을 주도하고 있다’고 말했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 671

In [109]:
run_summarize(sentence, models, max_tokens=300, min_tokens=200, num_beams=2) # do_sample = False, no_repeat = 3

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 이에 따라 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 

이에 약세를 보이고 있다.

김석환 미래에셋증권 연구원은 “TSMC가반도체 업황을 하향 조정한 여파로 밤사이 미국 증시에서 반도체 관련 종목군의 약세가 뚜렷했다”며 “한국 증시에서도 관련 종목군을 주가 약세가 지수 하락을 주도하고 있다” 고 했다. 

김약환 미래에셋증권은 김석환 티브엠을 비롯한 코스닥시장 시가총액 상위 종목 주식 대부분이 전날보다 낮은 가격에 거래 중이다
 


이에 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다.
(왼쪽(왼쪽(왼쪽) 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 코스피·코스닥지수가 약세를 보이고 있으며, 특히 고금리·고환율·저유가 등 이른바 3고 우려가 다시 불거지면서 19일장중 코스피와 코스닥지수는 약세 폭을 키우고 있고, 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등한 것으로 보인다.


한국 증시에서도 관련 종목군의 주가 약세가 지수 하락을 주도하고 있다.
한국의 코스피지수 역시 전날보다 67.93포인트(2.58%) 하락했다. 한국 주식시장 시가총액 상위 종목 주식 대부분이 전날 보다 낮은 가격에 거래 중이다.
미국 달러 대비 원화(원·달러) 환율은 이날 오전 1390원선을 

In [110]:
def summarize(input_text, tokenizer, model, device, max_tokens=200, min_tokens=100, num_beams=2) :
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Generate Summary Text Ids
    summary_text_ids = model.generate(
        input_ids=input_ids,
        bos_token_id=model.config.bos_token_id,
        eos_token_id=model.config.eos_token_id,
        length_penalty=2.0,
        max_new_tokens=max_tokens,
        min_new_tokens=min_tokens,
        no_repeat_ngram_size=3,
        num_beams=num_beams,
        top_p=0.95,
        do_sample=False #True
    )

    response = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)

    #print(len(input_text), len(response))

    return response

In [111]:
run_summarize(sentence, models, max_tokens=300, min_tokens=200, num_beams=2) # do_sample = False, no_repeat = 3

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 이에 따라 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 

이에 약세를 보이고 있다.

김석환 미래에셋증권 연구원은 “TSMC가반도체 업황을 하향 조정한 여파로 밤사이 미국 증시에서 반도체 관련 종목군의 약세가 뚜렷했다”며 “한국 증시에서도 관련 종목군을 주가 약세가 지수 하락을 주도하고 있다” 고 했다. 

김약환 미래에셋증권은 김석환 티브엠을 비롯한 코스닥시장 시가총액 상위 종목 주식 대부분이 전날보다 낮은 가격에 거래 중이다
 


이에 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다.
(왼쪽(왼쪽(왼쪽) 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 코스피·코스닥지수가 약세를 보이고 있으며, 특히 고금리·고환율·저유가 등 이른바 3고 우려가 다시 불거지면서 19일장중 코스피와 코스닥지수는 약세 폭을 키우고 있고, 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등한 것으로 보인다.


한국 증시에서도 관련 종목군의 주가 약세가 지수 하락을 주도하고 있다.
한국의 코스피지수 역시 전날보다 67.93포인트(2.58%) 하락했다. 한국 주식시장 시가총액 상위 종목 주식 대부분이 전날 보다 낮은 가격에 거래 중이다.
미국 달러 대비 원화(원·달러) 환율은 이날 오전 1390원선을 

In [114]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=2) # do_sample = False, no_repeat = 3

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 이에 따라 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 코스피·코스닥지수가 약세를 보이고 있으며, 특히 고금리·고환율·저유가 등 이른바 3고 우려가 다시 불거지면서 19일장중 코스피와 코스닥지수는 약세 폭을 키우고 있다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸고 유가증권시장 대장주인 삼성전자와SK하이닉스가 각각 3%, 5% 이상 약세를 나타내고 있다. 코스피지수는 이날 오전 11시 4분 기준 2566.77이다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있으며, 특히 코스피지수는 이날 오전 11시 4분 기준 2566.77로 전날보다 67.93포인트(2.57%) 하락한 것으로 나타났다.

In [115]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=3) # do_sample = False, no_repeat = 3

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다. 

중동 정세 불안으로 국제 유가도 뛰었다. 

이스라엘이 계획 중인 가자지구 남단 라파 지상전을 두고 미국 백악관은 민간인 피해 가능성을 여전히 우려하지만, 팔레스타인 무장 정파 하마스를 격퇴한다는 공동의 목표에 합의했다고 발표했다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 코스피·코스닥지수가 약세를 보이고 있으며, 특히 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 김석환 미래에셋증권 연구원은 한국 증시에서도 관련 종목군의 주가 약세가 지수 하락을 주도하고 있다고 했다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 

In [116]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=4) # do_sample = False, no_repeat = 3

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 이에 따라 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체업종 주가가 꺾이면서 코스피·코스닥지수가 약세를 보이고 있으며 특히 고금리·고환율,고유가 등의 이른바 3고 우려가 다시 불거지면서 18일 이후 가장 낮은 수준으로 하락 폭을 키우고 있는 가운데 코스피지수는 이날 오전 11시 4분 기준 2566.77로 전날보다 67.93포인트(2.58%) 하락했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 김석환 미래에셋증권 연구원은 한국 증시에서도 관련 종목군의 주가 약세가 지수 하락을 주도하고 있다고 했다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있으며, 특히 코스피지수는 이날 오전 11시 4분 기준 25

In [117]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=8) # do_sample = False, no_repeat = 3

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 이에 따라 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 

====  noahkim/KoT5_news_summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 올해 메모리 반도체를 제외한 전체 반도체 시장 성장률 전망치를 기존 ‘10% 이상’에서 10%로 낮춘 영향으로 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데, 김석환 미래에셋증권 연구원은 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 코스피지수는 이날 오전 11시 4분 기준 2566.77이며 전날보다 67.93포인트(2.58%) 하락했다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있으며, 특히 코스피지수는 이날 오전 11시 4분 기준 2566.77로 전날보다 67.93포인트(2.57

In [118]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=16) # do_sample = False, no_repeat = 3

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 이에 따라 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 

====  noahkim/KoT5_news_summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 올해 메모리 반도체를 제외한 전체 반도체 시장 성장률 전망치를 기존 ‘10% 이상’에서 10%로 낮추는 등 이른바 3고 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데, 김석환 미래에셋증권 연구원은 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다. 

====  psyche/KoT5-summarization  ===
고금리·고환율·고유가 등 이른바 3고() 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
19일 장 중 코스피·코스닥지수가 약세를 보이고 있는데, 특히 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며, 코스피지수는 이날 오전 11시 4분 기준 2566.

In [119]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=32) # do_sample = False, no_repeat = 3

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 이에 따라 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 

====  noahkim/KoT5_news_summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 올해 메모리 반도체를 제외한 전체 반도체 시장 성장률 전망치를 기존 ‘10% 이상’에서 10%로 낮춘 영향으로 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등하는 등 이른바 3고 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 

====  psyche/KoT5-summarization  ===
고금리·고환율·고유가 등 이른바 3고() 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다. 중동 정세 불안으로 국제 유가도 뛰었다. 이스라엘이 계획 중인 가자지구 남단 라파 지상전을 두고 미국 백악관은 민간인 피해 가능성을 여전히 우려하지만, 팔레스타인 무장 정파 하마스를 격퇴한다는 공동의 목표에 합의했다고 발표했다. 이어 이스라엘이 이란 본토를 미사일로 공격했다는 보도도 나왔다. 미국 서부텍사스원유(WTI) 5월물과 브렌

In [148]:
def summarize(input_text, tokenizer, model, device, max_tokens=200, min_tokens=100, num_beams=2) :
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Generate Summary Text Ids
    summary_text_ids = model.generate(
        input_ids=input_ids,
        bos_token_id=model.config.bos_token_id,
        eos_token_id=model.config.eos_token_id,
        length_penalty=2.0,
        max_new_tokens=max_tokens,
        min_new_tokens=min_tokens,
        no_repeat_ngram_size=3,
        num_beams=num_beams,
        top_p=0.95, # 누적 확률이 top_p %인 후보집합에서만 생성
        top_k=50, # 확률 순위 top_k 밖의 sample은 제외
        do_sample=True
    )

    response = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)

    #print(len(input_text), len(response))

    return response

In [131]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=32) # top_p = 0.95, top_k = 0

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 이에 따라 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 코스피·코스닥지수가 약세를 보이고 있으며 이에 김석환 미래에셋증권 연구원은 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등한 것은 중동 정세 불안으로 국제 유가도 뛰었고 이스라엘이 계획 중인 가자지구 남단 라파 지상전을 두고 백악관은 민간인 피해 가능성을 여전히 우려하지만, 팔레스타인 무장 정파 하마스를 격퇴한다는 공동의 목표에 합의했다고 발표했다. 뉴욕 증시에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸고, 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다. 중동 정세 불안으로 국제 유가도 뛰었다. 이스라엘이 계획 중인 가

In [133]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=8) # do_sample = False, no_repeat = 3

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 이에 따라 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 코스피·코스닥지수가 약세를 보이고 있으며, 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 코스피지수는 이날 오전 11시 4분 기준 2566.77이며 전날보다 67.93포인트(2.58%) 하락했다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있으며, 특히 코스피지수는 이날 오전 11시 4분 기준 2566.77로 전날보다 67.93포인트(2.57%) 하락한 것으로 나타났으며, 이는 지난 1월 이후 가장 낮은 수준이다. 



In [135]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=32) # top_k = 20

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 이에 따라 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 코스피·코스닥지수가 약세를 보이고 있으며 이에 김석환 미래에셋증권 연구원은 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등한 것은 중동 정세 불안으로 국제 유가도 뛰었고 이스라엘이 계획 중인 가자지구 남단 라파 지상전을 두고 백악관은 민간인 피해 가능성을 여전히 우려하지만, 팔레스타인 무장 정파 하마스를 격퇴한다는 공동의 목표에 합의했다고 발표했다. 뉴욕 증시에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 한국 증시에서도 관련 종목군의 주가 약세가 지수 하락을 주도하고 있다. 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 8 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸고, 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다. 중동 정세 불안

In [136]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=8) # top_k = 20

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있어 코스피지수는 이날 오전 11시 4분 기준 2566.77로 전날보다 67.93포인트(2.58%) 하락했다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 코스피·코스닥지수가 약세를 보이고 있으며 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 고금리·고환율·고유가 등 이른바 3고() 우려가 다시 불거지면서 19일 오전 11시 4분 기준 2566.77로 전날보다 67.93포인트(2.58%) 하락하며 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 김석환 미래에셋증권 연구원은 반도체 업황을 하향 조정한 여파로 밤사이 미국 증시에서 반도체 관련 종목군의 약세가 뚜렷했다며 한국 증시에서도 관련종목군의 주가 약세가 지수 하락을 주도하고 있다고 했다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
19일 장 중 코스피·코스닥지수가 약세를 보이고 있는데, 특히 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성

In [137]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=8) # top_k = 20

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록하며 전날 상승분을 모두 반납했다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 코스피·코스닥지수가 약세를 보이고 있으며, 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다. 중동 정세 불안으로 국제 유가도 뛰었다. 이스라엘이 계획 중인 가자지구 남단 라파 지상전을 두고 미국 백악관은 민간인 피해 가능성을 여전히 우려하지만, 팔레스타인 무장 정파 하마스를 격퇴한다는 공동의 목표에 합의했다고 발표했다. 이어 이스라엘이 이란 본토를 미사일로 공격했다는 보도도 나왔다. 미국 서부텍사스원유(WTI) 5월물과 브렌트유 6월물 모두 현재 3%대 오름세를 나타내고 있다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
19일 장

In [138]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=8) #  top_k = 20

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 이에 따라 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 코스피·코스닥지수가 약세를 보이고 있으며 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 김석환 미래에셋증권 연구원은 한국 증시에서도 관련 종목군의 주가 약세가 지수 하락을 주도하고 있다고 했다. 중동 정세 불안으로 국제 유가도 뛰었다. 이스라엘이 계획 중인 가자지구 남단 라파 지상전을 두고 미국 백악관은 민간인 피해 가능성을 여전히 우려하지만, 팔레스타인 무장 정파 하마스를 격퇴한다는 공동의 목표에 합의했다고 발표했다. 이어 이스라엘이 이란 본토를 미사일로 공격했다는 보도도 나왔다. 미국 서부텍사스원유(WTI) 5월물과 브렌트유 6월물 모두 현재 3%대 오름세를 나타내고 있다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summar

In [140]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=8) #  top_k = 20, top_p = 0.95

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록하며 전날 상승분을 모두 반납했다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 코스피·코스닥지수가 약세를 보이고 있으며 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸고, 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다. 중동 정세 불안으로 국제 유가도 뛰었다. 이스라엘이 계획 중인 가자지구 남단 라파 지상전을 두고 미국 백악관은 민간인 피해 가능성을 여전히 우려하지만, 팔레스타인 무장 정파 하마스를 격퇴한다는 공동의 목표에 합의했다고 발표했다. 이어 이스라엘이 이란 본토를 미사일로 공격했다는 보도도 나왔다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
19일 장 중 코스피·코스닥지수가 약세를 보이고 있는데, 특히 세계 최대 파운드리 업체인 

In [141]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=8) #  top_k = 20, top_p = 0.95

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다. 
. 하락하며 하루 만에 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다.

또 이스라엘이 계획 중인 가자지구 남단 라파 지상전을 두고 미국 백악관은 민간인 피해 가능성을 우려하지만, 팔레스타인 무장 정파 하마스를 격퇴한다는 공동의 목표에 합의했다고 발표했다라는 발표와 이스라엘이 이란 본토를 미사일로 공격했다는 보도도 나오면서 중동 정세 불안으로 국제 유가도 뛰었다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 코스피·코스닥지수가 약세를 보이고 있으며, 특히 고금리·고환율, 고유가 등 이른바 3고 우려가 다시 불거지면서 19일, 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸고, 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다. 중동 정세 불안으로 

In [142]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=8) #  top_k = 20, top_p = 0.95

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 이에 따라 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 코스피·코스닥지수가 약세를 보이고 있으며 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 코스피지수는 이날 오전 11시 4분 기준 2566.77으로 전날보다 67.93포인트(2.58%) 하락했다. 코스피지수와 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했으며 코스닥시장 시가총액 상위 종목 주식 대부분이 전날보다 낮은 가격에 거래 중이다. 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다. 중동 정세 불안으로 국제 유가도 뛰었다. 이스라엘이 계획 중인 가자지구 남단 라파 지상전을 두고 미국 백악관은 민간인 피해 가능성을 여전히 우려하지만, 팔레스타인 무장 정파 하마스를 격퇴한다는 공동의 목표에 합의했다고 발표

In [146]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=8) #  top_k = 20, top_p = 0.95

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록하며 전날 상승분을 모두 반납했다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 코스피·코스닥지수가 약세를 보이고 있으며 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 코스피지수는 이날 오전 11시 4분 기준 2566.77이며 전날보다 67.93포인트(2.58%) 하락했다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
19일 장 중 코스피·코스닥지수가 약세를 보이고 있는데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고 코스닥시장에서도 외국인과 기관이 3185억원, 330억원을 순매도하고 있어 지수를 방어하기엔 벅찬 상황이며 미국 국채 2년

In [147]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=8) #  top_k = 20, top_p = 0.95

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다. 

중동 정세 불안으로 국제 유가도 뛰었으며 특히 이스라엘이 계획 중인 가자지구 남단 라파 지상전을 두고 미국 백악관은 민간인 피해 가능성을 우려하지만, 팔레스타인 무장 정파 하마스를 격퇴한다는 공동의 목표에 합의했다고 발표했다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 코스피·코스닥지수가 약세를 보이고 있으며, 특히 고금리·고환율,고유가 등 이른바 3고 우려가 다시 불거지면서 19일, 코스피지수는 이날 오전 11시 4분 기준 2566.77로 전날보다 67.93포인트(2.58%) 하락했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다. 중동 정세 불안으로 국제 유가도 뛰었다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
19일 장 중 코스피·코스닥지수가 약세를 보이고 있는데, 특히 세계 

In [149]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=8) #  top_k = 50, top_p = 0.95

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 이에 따라 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 코스피·코스닥지수가 약세를 보이고 있으며, 특히 고금리·고환율·저유가 등 이른바 3고 우려가 다시 불거지면서 19일, 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등한 것은 중동 정세 불안으로 국제 유가도 뛰었다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸고, 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다. 중동 정세 불안으로 국제 유가도 뛰었다. 이스라엘이 계획 중인 가자지구 남단 라파 지상전을 두고 미국 백악관은 민간인 피해 가능성을 여전히 우려하지만, 팔레스타인 무장 정파 하마스를 격퇴한다는 공동의 목표에 합의했다고 발표했다. 이어 이스라엘이 이란 본토를 미사일로 공격했다는 보도도 나왔다. 미국 서부텍사스원유(WTI) 5월물과 브렌트유 6월물 모두 현재 3%대 오름세를 보이고 있는

In [150]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=8) #  top_k = 50, top_p = 0.95

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 이에 따라 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 코스피·코스닥지수가 약세를 보이고 있으며, 특히 고금리·고환율·저유가 등 이른바 3고 우려가 다시 불거지면서 19일장 내에서 코스피지수와 코스닥지수의 약세 폭을 키우고 있고, 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등한 것은 중동 정세 불안으로 국제 유가도 뛰었고, 이스라엘이 계획 중인 가자지구 남단 라파 지상전을 두고 백악관은 민간인 피해 가능성을 여전히 우려하지만, 팔레스타인 무장 정파 하마스를 격퇴한다는 공동의 목표에 합의했다고 발표했다. 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했으며, 코스닥시장 시가총액 상위 종목 주식 대부분이 전날보다 낮은 가격 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데, 김석환 미래에셋증권 연구원은 "한국 증시에서도 관련 종목군의 주가 약세가 지수 하락을 주도하고 있다"고 했고, 코스닥시장에서도 외국인과 기관이 3185억원, 330억원을 

In [154]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=4) #  top_k = 50, top_p = 0.95

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 이에 따라 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 코스피·코스닥지수가 약세를 보이고 있으며 이에 김석환 미래에셋증권 연구원은 “미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다”고 말했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 코스피지수는 이날 오전 11시 4분 기준 2566.77이며 전날보다 67.93포인트(2.58%) 하락했다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데, 코스피지수는 이날 오전 11시 4분 기준 2566.77로 전날보다 67.93포인트(2.57%) 하락한 것으로 나타났으며, 이는 지

In [155]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=8) #  top_k = 50, top_p = 0.95

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 이에 따라 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 코스피·코스닥지수가 약세를 보이고 있으며, 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 김석환 미래에셋증권 연구원은 한국 증시에서도 관련 종목군의 주가 약세가 지수 하락을 주도하고 있다고 했다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
19일 장 중 코스피·코스닥지수가 약세를 보이고 있는데, 특히 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며, 코스닥시장에서도 외국인과 기관이 3185억원, 330억원을 순매도하고 있어 지수를 방어하기엔 벅찬 상황이다.  



In [156]:
def summarize(input_text, tokenizer, model, device, max_tokens=200, min_tokens=100, num_beams=2) :
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Generate Summary Text Ids
    summary_text_ids = model.generate(
        input_ids=input_ids,
        bos_token_id=model.config.bos_token_id,
        eos_token_id=model.config.eos_token_id,
        length_penalty=2.0,
        max_new_tokens=max_tokens,
        min_new_tokens=min_tokens,
        no_repeat_ngram_size=3,
        num_beams=num_beams,
        top_p=0.92, # 누적 확률이 top_p %인 후보집합에서만 생성
        top_k=50, # 확률 순위 top_k 밖의 sample은 제외
        do_sample=True
    )

    response = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)

    #print(len(input_text), len(response))

    return response

In [157]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=8) #  top_k = 50, top_p = 0.95

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록하며 전날 상승분을 모두 반납했다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 코스피·코스닥지수가 약세를 보이고 있으며 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸고, 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다. 중동 정세 불안으로 국제 유가도 뛰었다. 이스라엘이 계획 중인 가자지구 남단 라파 지상전을 두고 미국 백악관은 민간인 피해 가능성을 여전히 우려하지만, 팔레스타인 무장 정파 하마스를 격퇴한다는 공동의 목표에 합의했다고 발표했다. 이어 이스라엘이 이란 본토를 미사일로 공격했다는 보도도 나왔다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
19일 장 중 코스피·코스닥지수가 약세를 보이고 있는데, 특히 세계 최대 파운드리 업체인 

In [158]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=4) #  top_k = 50, top_p = 0.95

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록하며 전날 상승분을 모두 반납했다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 올해 메모리 반도체를 제외한 전체 반도체 시장 성장률 전망치를 기존 ‘10% 이상’에서 10%로 낮춘 영향으로 코스피·코스닥지수가 약세를 보이고 있는 가운데 김석환 미래에셋증권 연구원은 미국 국채 2년물 금리가 5%에 육박하는 가운데, 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 김석환 미래에셋증권 연구원은 한국 증시에서도 관련 종목군의 주가 약세가 지수 하락을 주도하고 있다고 했다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는데, 이는 지난 1월 이후 가장 낮은 수준이며, 코스피지수는 이날 오전 11시 4분 기준 2566.77로 전날

In [160]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=4) #  top_k = 50, top_p = 0.95

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 이에 따라 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 코스피·코스닥지수가 약세를 보이고 있으며 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸고, 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다. 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있으며 이는 지난 1월 이후 가장 낮은 수준으로 코스피지수는 이날 오전 11시 4분 기준 2566.77로 전날보다 67.9

In [162]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=8) #  top_k = 50, top_p = 0.95

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다. 

중동 정세 불안으로 국제 유가도 뛰었으며 특히 이스라엘이 계획 중인 가자지구 남단 라파 지상전을 두고 미국 백악관은 민간인 피해 가능성을 우려하지만, 팔레스타인 무장 정파 하마스를 격퇴한다는 공동의 목표에 합의했다고 발표했다. 

====  noahkim/KoT5_news_summarization  ===
19일 장 중 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자 반도체와 반도체장비의 하락률이 3.8%로 커지면서 코스피·코스닥지수가 약세를 보이고 있으며 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 코스피지수는 이날 오전 11시 4분 기준 2566.77이며 전날보다 67.93포인트(2.58%) 하락했다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
19일 장 중 코스피·코스닥지수가 약세를 보이고 있는데, 특히 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉

In [167]:
def summarize(input_text, tokenizer, model, device, max_tokens=200, min_tokens=100, num_beams=2) :
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Generate Summary Text Ids
    summary_text_ids = model.generate(
        input_ids=input_ids,
        bos_token_id=model.config.bos_token_id,
        eos_token_id=model.config.eos_token_id,
        length_penalty=2.0,
        max_new_tokens=max_tokens,
        min_new_tokens=min_tokens,
        no_repeat_ngram_size=3,
        num_beams=num_beams,
        #top_p=0.92, # 누적 확률이 top_p %인 후보집합에서만 생성
        #top_k=50, # 확률 순위 top_k 밖의 sample은 제외
        do_sample=False # Greedy
    )

    response = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)

    #print(len(input_text), len(response))

    return response

In [168]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=8) #  top_k = 50, top_p = 0.95

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 이에 따라 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 

====  noahkim/KoT5_news_summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 올해 메모리 반도체를 제외한 전체 반도체 시장 성장률 전망치를 기존 ‘10% 이상’에서 10%로 낮춘 영향으로 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데, 김석환 미래에셋증권 연구원은 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다. 

====  psyche/KoT5-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 코스피지수는 이날 오전 11시 4분 기준 2566.77이며 전날보다 67.93포인트(2.58%) 하락했다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있으며, 특히 코스피지수는 이날 오전 11시 4분 기준 2566.77로 전날보다 67.93포인트(2.57

In [169]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=16) #  top_k = 50, top_p = 0.95

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 이에 따라 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 

====  noahkim/KoT5_news_summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 올해 메모리 반도체를 제외한 전체 반도체 시장 성장률 전망치를 기존 ‘10% 이상’에서 10%로 낮추는 등 이른바 3고 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데, 김석환 미래에셋증권 연구원은 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다. 

====  psyche/KoT5-summarization  ===
고금리·고환율·고유가 등 이른바 3고() 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
19일 장 중 코스피·코스닥지수가 약세를 보이고 있는데, 특히 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며, 코스피지수는 이날 오전 11시 4분 기준 2566.

In [170]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=32) #  top_k = 50, top_p = 0.95

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 이에 따라 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 

====  noahkim/KoT5_news_summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 올해 메모리 반도체를 제외한 전체 반도체 시장 성장률 전망치를 기존 ‘10% 이상’에서 10%로 낮춘 영향으로 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등하는 등 이른바 3고 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 

====  psyche/KoT5-summarization  ===
고금리·고환율·고유가 등 이른바 3고() 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 같은 시각 코스닥지수도 24.47포인트(2.86%) 내린 831.18을 기록했다. 중동 정세 불안으로 국제 유가도 뛰었다. 이스라엘이 계획 중인 가자지구 남단 라파 지상전을 두고 미국 백악관은 민간인 피해 가능성을 여전히 우려하지만, 팔레스타인 무장 정파 하마스를 격퇴한다는 공동의 목표에 합의했다고 발표했다. 이어 이스라엘이 이란 본토를 미사일로 공격했다는 보도도 나왔다. 미국 서부텍사스원유(WTI) 5월물과 브렌

In [171]:
run_summarize(sentence, models, max_tokens=300, min_tokens=100, num_beams=16) #  top_k = 50, top_p = 0.95

====  ainize/kobart-news  ===
파금리·고환율·고유가 등 이른바 3고(高) 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보여, 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있고, 이에 따라 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 ‘팔자’에 나서며 지수를 끌어내렸다. 

====  noahkim/KoT5_news_summarization  ===
세계 최대 파운드리 업체인 대만 TSMC가 올해 메모리 반도체를 제외한 전체 반도체 시장 성장률 전망치를 기존 ‘10% 이상’에서 10%로 낮추는 등 이른바 3고 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있는 가운데, 김석환 미래에셋증권 연구원은 미국 국채 2년물 금리가 5%에 육박하는 가운데 외환 당국의 구두개입으로 주춤했던 환율도 하루 만에 반등했다. 

====  psyche/KoT5-summarization  ===
고금리·고환율·고유가 등 이른바 3고() 우려가 다시 불거지면서 19일 장 중 코스피·코스닥지수가 약세를 보이고 있다. 특히 세계 최대 파운드리(반도체 위탁생산) 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있다. 유가증권시장에서 기관과 외국인이 각각 6712억원, 2743억원 팔자에 나서며 지수를 끌어내렸다. 

====  KETI-AIR-Downstream/long-ke-t5-base-summarization  ===
19일 장 중 코스피·코스닥지수가 약세를 보이고 있는데, 특히 세계 최대 파운드리 업체인 대만 TSMC가 반도체 성장 전망치를 낮추자, 삼성전자와 SK하이닉스 등 반도체 업종 주가가 꺾이면서 지수 하락 폭을 키우고 있으며, 코스피지수는 이날 오전 11시 4분 기준 2566.